In [1]:
%%HTML
<link rel="stylesheet" type="text/css" href="./static/css/custom.css">
<link rel="stylesheet" type="text/css" href="./static/css/highlight.css">

In [2]:
import datetime
from io import StringIO

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import FloatSlider, IntSlider, interact

%matplotlib inline
%load_ext tsumiki

## ipywidgets

#### ipywidgets
* Notebook上にインタラクティブなUIを提供
* 多数のインタフェース  
  Button, Slider, Dropdown, RadioButtons, Date picker...
* 実装が容易

#### 3行でできるUI

In [3]:
@interact(checked=True, selected=["A", "B", "C"])
def f(checked, selected):
    return checked, selected

interactive(children=(Checkbox(value=True, description='checked'), Dropdown(description='selected', options=('…

#### 事例:ファンドの組み入れ比率
* 複数の投資信託の投資比率を考えてみる  
  ファンドA, ファンドB, ファンドC
* できるだけリターンを大きくしたい
* できるだけリスクを小さくしたい

#### 投資比率をパラメータ化
1. リターンとリスクを算出する関数を作成
2. パラメータを変更して`1.`の結果を確認

* パラメータを手入力で変更するのは大変
* すべての組み合わせを一覧化・可視化すると情報が多すぎる
* 状況に応じて裁量でパラメータを調整したい

#### ipywidgetsでインタラクティブにリターンとリスクを可視化

In [4]:
def fund_returns(s0, mu, sigma, t, dt):
    return s0 * np.exp(
        np.cumsum(np.random.normal(mu * dt, sigma * np.sqrt(dt), round(t / dt)))
    )


def get_return_and_risk(a, b):
    weight = (a, b, 1 - sum((a, b)))
    return_df = df * weight
    return return_df.sum(axis=1).pct_change().var(), return_df.iloc[-1].sum() / 100


np.random.seed(9)
fund_a = fund_returns(100, 0.1, 0.15, 1, 0.01)
fund_b = fund_returns(100, 0.12, 0.4, 1, 0.01)
fund_c = fund_returns(100, 0.05, 0.1, 1, 0.01)
df = pd.DataFrame({"A": fund_a, "B": fund_b, "C": fund_c})


@interact(
    a=FloatSlider(0.3, min=0.0, max=1.0, step=0.1, continuous_update=False),
    b=FloatSlider(0.3, min=0.0, max=1.0, step=0.1, continuous_update=False),
)
def plot_return_and_risk(a, b):
    c = 1 - sum((a, b))
    fig = plt.figure(figsize=(14, 6))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(fund_a, label="FundA")
    ax1.plot(fund_b, label="FundB")
    ax1.plot(fund_c, label="FundC")
    ax2 = fig.add_subplot(1, 2, 2)
    ax1.legend()
    ax2.set_title(f"a:{a:.2}0%, b:{b:.2}0%, c:{c:.2}0%", fontsize=20)
    ax2.set_xlim(0, 0.002)
    ax2.set_xlabel("Risk", fontsize=20)
    ax2.set_ylabel("Return", fontsize=20)
    ax2.set_ylim(0.8, 1.5)
    ax2.scatter(*get_return_and_risk(a, b), s=1000)
    ax2.axvline(0.001, c="black")
    ax2.axhline(1.0, c="black")

interactive(children=(FloatSlider(value=0.3, continuous_update=False, description='a', max=1.0), FloatSlider(v…

## Jupyter Notebookでプレゼンテーション

#### 資料作成の悩み
1. 資料に使えそうなコードを書く
2. コードや実行結果をスライドに転記する
3. `1.`を途中で変更する
4. ウギャーとなる  
  - 転記が大変
  - 漏れが出る

#### Jupyter Notebook
* スライドに変換する機能がある(nbconvert)
* マークダウンで書ける
* コードも実行結果もそのままスライドにできる
* ウギャーとならない

#### Jupyter Notebookでいいんじゃね？
* 変換がメンドウ
* 変換後はコードが実行できない

#### RISE
* Reveal.jsを利用したnbextensions
* ボタン1発でNotebookからスライドに
* スタイルをカスタマイズできる
* スライド上からコードを実行できる
* nbpresentというツールもあるらしい

#### 再掲

In [5]:
@interact(checked=True, selected=["A", "B", "C"])
def f(checked, selected):
    return checked, selected

interactive(children=(Checkbox(value=True, description='checked'), Dropdown(description='selected', options=('…

RISEを使うとインタラクティブなプレゼンが可能に

#### コードが邪魔
* 大抵において、プレゼン資料で必用なのはコードではなく実行結果  
  - テーブルデータ(pandas.DataFrameなど)
  - グラフ(Matplotlibなど)
* 隠せるものなら隠したい  
  - 全部のコード
  - 一部のコード

#### Hide_code
* nbextensions
* チェックした項目を隠せる  
  - [ ] Hide Prompts
  - [ ] Hide Code
  - [ ] Hide Outeputs

#### cellの内容を固定したい
* コードは見せたいが実行したくない
* 一度実行した結果を固定したい

<i class="fa-asterisk fa"></i>  Freezeを用いてfrozenにする

In [6]:
%%tsumiki

:Markdown:
#### 段組みにしたい

:Markdown::
* よのなかのアスペクト比はどんどん横長に
* プレゼン資料で横長の文章は読みづらい
* 左右に分割して比較をしたい

:Markdown::
* ここには
* 有効なスペースが
* ありますよ

#### tsumiki
* どりらんが考案したマークアップ記法
* MarkdownやHTMLで記載した内容を複数列に表示できる
* Jinja2に対応

In [ ]:
%%tsumiki

:Markdown:
# Main Title1
:Markdown::
* col1
* Markdown
:HTML::
<font color="red">col2</font></br>
<font color="green">HTML</font>
:Markdown:
## Subtitle
:Markdown:::
* col1
:Markdown:::
* col2
:Markdown:::
* col3

In [7]:
%%tsumiki

:Markdown:
# Main Title1
:Markdown::
* col1
* Markdown
:HTML::
<font color="red">col2</font></br>
<font color="green">HTML</font>
:Markdown:
## Subtitle
:Markdown:::
<style>
.tsumiki #col1 { width: 33% !important}
</style>
* col1
:Markdown:::
* col2
:Markdown:::
* col3

#### Jinja2に対応

In [8]:
today = datetime.date.today()
tenki = [["23区", "晴れ"], ["練馬区", "ゲリラ豪雨"]]

In [9]:
%%tsumiki -r

:Markdown:
### {{ today }}日の天気は
{% for loc, weather in tenki %}
  {{ loc }}は{{ weather }}
{% endfor %}

#### pandas.DataFrame

In [10]:
df1 = pd.DataFrame(np.random.rand(4, 4))
df2 = pd.DataFrame(np.random.rand(4, 4))
df1_html = df1.to_html()
df2_html = df2.to_html()

In [11]:
%%tsumiki -r

:HTML::
{{ df1_html }}

:HTML::
{{ df2_html }}

,0,1,2,3
0,0.619147,0.189785,0.935151,0.708119
1,0.801716,0.780755,0.523305,0.631772
2,0.018740,0.583013,0.609735,0.667796
3,0.203620,0.320793,0.973482,0.273809
,0,1,2,3
0,0.699981,0.265305,0.572697,0.872142
1,0.817083,0.029230,0.424361,0.393610
2,0.900326,0.035806,0.454794,0.981336
3,0.036405,0.670754,0.734716,0.104887


#### Matplotlib

In [15]:
svg_data = StringIO()
fig, ax = plt.subplots()
ax.plot(df1)
fig.savefig(svg_data, format="SVG")
plt.close(fig)
svg_data.seek(0)
plot_data = svg_data.read()

In [16]:
%%tsumiki -r

:Markdown:
##### DataFrame and Plot

:HTML::
{{ df1_html }}
:HTML::
{{ plot_data }}

,0,1,2,3
0,0.619147,0.189785,0.935151,0.708119
1,0.801716,0.780755,0.523305,0.631772
2,0.018740,0.583013,0.609735,0.667796
3,0.203620,0.320793,0.973482,0.273809


## Jupyter Publishing

#### 技術的な情報を
* Webサイトにまとめたい
* ブログの記事として投稿したい
* ドキュメントを書きたい
* 技術書を書きたい

#### 技術的な情報をまとめるのに大変なこと

* コードとその解説のフォーマットが異なる
* コードを記事や原稿に転記するのが大変
* コードと情報のファイルが別々で管理が煩雑
* 原稿に書いてあるコードのテストが難しい

#### Jupyter Notebookなら全部できる
* 解説部分を軽量マークアップ言語で記述できて
  - 表や図、画像などの埋め込み
  - 数式を表現
  - raw HTML
* 記述したコードを実行できて
* 実行したコードの出力を保存できる

#### 必用なもの

* Jupyter Notebook
* サイトジェネレータ or ドキュメントビルダー

That's All!

#### 静的サイトジェネレータ・ドキュメントツール

* Shpinx  
  nbsphinxプラグイン
* Pelican  
  pelican-ipynbプラグイン
* Nikola    
  設定のみ
* Miyadaiku  
  設定のみ

#### 実際にJupyter Notebookでかいたもの
* ブログ  
  Miyadaikuを使用  
  https://drillan.github.io
* 雑誌の記事  
  nbsphinxを使用  
  Software Design 2018年2月号

## 便利なツール・サービス

#### よくいわれること
「Jupyterってdiffがなぁ・・・」

```bash
$ diff plot1.ipynb plot2.ipynb 
22c22
<        "[<matplotlib.lines.Line2D at 0x7f1ecc596710>]"
---
>        "[<matplotlib.lines.Line2D at 0x7fb2184d9748>]"
31c31
<       "image/png": "iVBORw0KGgoAAAANSUhEUgAAAXcAAAD8CAYAAACMwORRAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDIuMi4yLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvhp/UCwAAIABJREFUeJzt3XlcVXX+x/HXVwUVRFzAFRE33EBLUbOasmxKzRazmWraraym2X+TmtZo2WI2M00zLY41LU5NzSS4pGWrZYtWWsmmKOKGGygKCLLe7+8P+P0eTqlc9cDhnvt+Ph48As6J8/564e3x3MPnGmstIiLiLU3cDiAiIs5TuYuIeJDKXUTEg1TuIiIepHIXEfEglbuIiAep3EVEPEjlLiLiQSp3EREPaubWgaOiomxcXJxbhxcRCUjr1q3bb6 ...
```

#### nbdime

* Notebookのdiffとmergeをやりやすくしたライブラリ
* gitコマンドに対応
* nbextensionを使うとボタン一発で履歴管理

#### nbdiff
```bash
$ nbdiff plot1.ipynb plot2.ipynb
--- plot1.ipynb  2018-09-09 18:10:56.678277
+++ plot2.ipynb  2018-09-09 18:11:11.428336
## modified /cells/1/outputs/0/data/text/plain:
-  [<matplotlib.lines.Line2D at 0x7f1ecc596710>]
+  [<matplotlib.lines.Line2D at 0x7fb2184d9748>]

## inserted before /cells/1/outputs/1:
+  output:
+    output_type: display_data
+    data:
+      image/png: iVBORw0K...<snip base64, md5=6d291fb88d778210...>
+      text/plain: <Figure size 432x288 with 1 Axes>


```

#### nbdiff(続き)
```bash
## deleted /cells/1/outputs/1:
-  output:
-    output_type: display_data
-    data:
-      image/png: iVBORw0K...<snip base64, md5=f287ce2ad0448575...>
-      text/plain: <Figure size 432x288 with 1 Axes>

## modified /cells/1/source:
@@ -1,2 +1,2 @@
 fig, ax = plt.subplots()
-ax.plot([1, 2])

+ax.plot([1, 2], color="red")
```

#### nbdiff-web
![nbdiff-web](static/images/nbdiff-web.png)

## ホスティング
GitホスティングサービスでNotebookがそのまま表示できる
* GitHub
* GitLab
* BitBucket

#### nbviewer

* NotebookをレンダリングするWebアプリケーション
* ブラウザがあればNotebookの表示ができる
* JavaScriptが使われたコードも表示できる
* スライド表示に対応
* binderへのリンク
* 独自のnbviewerサーバーをホストできる

In [4]:
%%tsumiki

:Markdown:
#### GitHub vs nbviewer

:HTML::
<h5>GitHub</h5>
<img src="static/images/render_github.png">

:HTML::
<h5>nbviewr</h5>
<img src="static/images/render_nbviewer.png">